In [37]:
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [4]:
dept = spark.read.csv('dept.csv', header=True, inferSchema=True)
case = spark.read.csv('case.csv', header=True, inferSchema=True)
source = spark.read.csv('source.csv', header=True, inferSchema=True)

In [6]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



In [54]:
source.write.json("data/sources_json", mode="overwrite")

In [50]:
source.write.csv("data/sources_csv", mode="overwrite", header=True)

In [10]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [19]:
dept.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------
 dept_division          | 311 Call Center        
 dept_name              | Customer Service       
 standardized_dept_name | Customer Service       
 dept_subject_to_SLA    | YES                    
-RECORD 1----------------------------------------
 dept_division          | Brush                  
 dept_name              | Solid Waste Management 
 standardized_dept_name | Solid Waste            
 dept_subject_to_SLA    | YES                    
only showing top 2 rows



In [22]:
dept = dept.withColumn('dept_subject_to_SLA', F.col('dept_subject_to_SLA') == 'YES')

In [11]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [23]:
case.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [39]:
case = case.withColumn('case_late', col('case_late') == 'YES')\
        .withColumn('case_closed', col('case_late') == 'YES')\
        .withColumn('case_id', format_string('%d', col('case_id')))\
        .withColumn('council_district', format_string('%d', col('council_district')))\
        .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))\
        .withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))\
        .withColumn('SLA_due_date', to_timestamp(col('SLA_due_date'), 'M/d/yy H:mm'))

In [32]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [13]:
source.show(2)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
+---------+----------------+
only showing top 2 rows



In [62]:
case.groupBy('case_status').count().show()

+-----------+------+
|case_status| count|
+-----------+------+
|       Open| 18110|
|     Closed|823594|
+-----------+------+



In [75]:
case.where((col('case_status') == 'Open'))\
    .select(max('num_days_late')).show()

+------------------+
|max(num_days_late)|
+------------------+
|       348.6458333|
+------------------+



In [84]:
case.where('num_days_late' == max('num_days_late'))\
    .show(vertical=True)

(0 rows)



In [45]:
case.where(col('service_request_type') == 'Stray Animal')\
    .count()

26760